In [46]:
import pandas as pd
from os import listdir
import json

In [47]:
def retrieve_concat_data(df, dir, label):
    with open(dir, 'r') as file:
        data = json.load(file)

    # Extract conversations
    conversations = [data[key] for key in data if key.startswith("conversation_")]
    example_index = data["example_index"]

    # Create the DataFrame with the same list of indices for each row
    tmp_df = pd.DataFrame({
        "line": conversations,
        "example_index": [example_index] * len(conversations)
    })
    tmp_df["label"] = label
    df = pd.concat([df, tmp_df], ignore_index=True)
    return df

In [48]:
true_train = pd.read_csv("../data/train_v2_400.csv", index_col = "index")
true_train["example_index"] = true_train.index

In [49]:
path = "../data/generated_patient_v1_200/"
patient_path = [path + f for f in listdir(path)]
try: patient_path.remove(".DS_Store")
except: pass

In [50]:
path = "../data/generated_control_v1_200/"
control_path = [path + f for f in listdir(path)]
try: control_path.remove(".DS_Store")
except: pass

In [51]:
complete_train = true_train.copy(deep=True)
for dir in patient_path:
    complete_train = retrieve_concat_data(complete_train, dir, label = 1)

for dir in control_path:
    complete_train = retrieve_concat_data(complete_train, dir, label = 0)

In [ ]:
record = set()
for idx, row in complete_train.iterrows():
    lines = row["line"].split("\n")
    cleaned_line = []
    for line in lines:
        record.add(line[:4])
        if line[:4] == "PAR:":
            cleaned_line.append(line[4:].strip())
    complete_train.at[idx, "line"] = "\n".join(cleaned_line)
print(record)

In [54]:
complete_train.to_csv("../data/train_complete_v1_800.csv")

In [55]:
true_test = pd.read_csv("../data/test_v2_149.csv", index_col = "index")
true_test["example_index"] = true_test.index

In [57]:
record = set()
for idx, row in true_test.iterrows():
    lines = row["line"].split("\n")
    cleaned_line = []
    for line in lines:
        record.add(line[:4])
        if line[:4] == "PAR:":
            cleaned_line.append(line[4:].strip())
    true_test.at[idx, "line"] = "\n".join(cleaned_line)
print(record)

{'PAR:', 'INV:'}


In [59]:
true_test.to_csv("../data/test_complete_v1_149.csv")